Задание 2

In [10]:
import sqlite3
import pandas as pd
import numpy as np

con = sqlite3.connect(':memory:')

# Загрузка и сохранение данных в SQLite
data = pd.read_csv('C:\\Users\\Roadmarshal\\Downloads\\boston.csv')
data.head()
data.to_sql('boston', con, index=False, if_exists='replace')

tasks = 0

# Задание 1: Подсчет количества пустых значений по выбранным колонкам

In [3]:
query_1 = '''
SELECT 'CRIM' AS column_name, COUNT(*) AS missing_values FROM boston WHERE CRIM IS NULL
UNION ALL
SELECT 'ZN', COUNT(*) FROM boston WHERE ZN IS NULL
UNION ALL
SELECT 'INDUS', COUNT(*) FROM boston WHERE INDUS IS NULL
UNION ALL
SELECT 'CHAS', COUNT(*) FROM boston WHERE CHAS IS NULL
UNION ALL
SELECT 'NOX', COUNT(*) FROM boston WHERE NOX IS NULL
'''
result_1 = pd.read_sql(query_1, con)
print(result_1)

tasks += 1

  column_name  missing_values
0        CRIM               0
1          ZN               0
2       INDUS               0
3        CHAS               0
4         NOX               0


# Задание 2: Вывод количества уникальных значений по колонкам CRIM, ZN, INDUS, CHAS, NOX

In [4]:
query_2 = '''
SELECT 'CRIM' AS column_name, COUNT(DISTINCT CRIM) AS unique_values FROM boston
UNION ALL
SELECT 'ZN', COUNT(DISTINCT ZN) FROM boston
UNION ALL
SELECT 'INDUS', COUNT(DISTINCT INDUS) FROM boston
UNION ALL
SELECT 'CHAS', COUNT(DISTINCT CHAS) FROM boston
UNION ALL
SELECT 'NOX', COUNT(DISTINCT NOX) FROM boston
'''
result_2 = pd.read_sql(query_2, con)
print(result_2)

tasks += 1

  column_name  unique_values
0        CRIM            504
1          ZN             26
2       INDUS             76
3        CHAS              2
4         NOX             81


# Задание 3: Вывод колонок, где медиана равна минимальному значению

In [5]:
query_3 = '''
SELECT column_name
FROM (
    SELECT 'CRIM' AS column_name, AVG(CRIM) AS median_value FROM (SELECT CRIM FROM boston ORDER BY CRIM LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
    UNION ALL
    SELECT 'ZN', AVG(ZN) FROM (SELECT ZN FROM boston ORDER BY ZN LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
    UNION ALL
    SELECT 'INDUS', AVG(INDUS) FROM (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
    UNION ALL
    SELECT 'CHAS', AVG(CHAS) FROM (SELECT CHAS FROM boston ORDER BY CHAS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
    UNION ALL
    SELECT 'NOX', AVG(NOX) FROM (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
) 
WHERE median_value = (SELECT MIN(median_value) FROM (
    SELECT AVG(CRIM) AS median_value FROM (SELECT CRIM FROM boston ORDER BY CRIM LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
    UNION ALL
    SELECT AVG(ZN) FROM (SELECT ZN FROM boston ORDER BY ZN LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
    UNION ALL
    SELECT AVG(INDUS) FROM (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
    UNION ALL
    SELECT AVG(CHAS) FROM (SELECT CHAS FROM boston ORDER BY CHAS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
    UNION ALL
    SELECT AVG(NOX) FROM (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2)
))
'''
result_3 = pd.read_sql(query_3, con)
print(result_3)

tasks += 1

  column_name
0          ZN
1        CHAS


# Задание 4: Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница). Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [6]:
query_highest_MEDV = '''
SELECT ROUND(AVG(RM)) as avg_rm_highest_MEDV, ROUND(AVG(MEDV), 2) as avg_MEDV_highest
FROM boston 
WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
'''
highest_MEDV = pd.read_sql(query_highest_MEDV, con).iloc[0]
avg_rm_highest_MEDV, avg_MEDV_highest = highest_MEDV
tasks += 1  # Увеличиваем счетчик задач после успешного выполнения запроса

quantities = [25, 50, 100, 200, 300]
for qty in quantities:
    query = f'''
    WITH ranked_boston AS (
        SELECT *, ROW_NUMBER() OVER (ORDER BY MEDV ASC) AS rank
        FROM boston
    )
    SELECT {qty} as qty_homes, ROUND(AVG(RM)) as avg_rm_lowest_MEDV, ROUND(AVG(MEDV), 2) as avg_MEDV_lowest
    FROM ranked_boston
    WHERE rank <= {qty}
    '''
    result = pd.read_sql(query, con).iloc[0]
    avg_rm_lowest_MEDV, avg_MEDV_lowest = result['avg_rm_lowest_MEDV'], result['avg_MEDV_lowest']
    difference = avg_rm_highest_MEDV - avg_rm_lowest_MEDV  # Разница в количестве комнат уже округлена до целого
    print(f"Для {qty} самых дешевых домов:")
    print(f"Среднее количество комнат (округлено до целого): {avg_rm_lowest_MEDV}")
    print(f"Средняя стоимость (округлено до сотых): {avg_MEDV_lowest}")
    print(f"Среднее количество комнат в самом дорогом доме (округлено до целого): {avg_rm_highest_MEDV}")
    print(f"Средняя стоимость самого дорогого дома (округлено до сотых): {avg_MEDV_highest}")
    print(f"Разница в среднем количестве комнат: {difference}\n")
    tasks += 1

Для 25 самых дешевых домов:
Среднее количество комнат (округлено до целого): 6.0
Средняя стоимость (округлено до сотых): 7.85
Среднее количество комнат в самом дорогом доме (округлено до целого): 7.0
Средняя стоимость самого дорогого дома (округлено до сотых): 50.0
Разница в среднем количестве комнат: 1.0

Для 50 самых дешевых домов:
Среднее количество комнат (округлено до целого): 6.0
Средняя стоимость (округлено до сотых): 9.65
Среднее количество комнат в самом дорогом доме (округлено до целого): 7.0
Средняя стоимость самого дорогого дома (округлено до сотых): 50.0
Разница в среднем количестве комнат: 1.0

Для 100 самых дешевых домов:
Среднее количество комнат (округлено до целого): 6.0
Средняя стоимость (округлено до сотых): 11.82
Среднее количество комнат в самом дорогом доме (округлено до целого): 7.0
Средняя стоимость самого дорогого дома (округлено до сотых): 50.0
Разница в среднем количестве комнат: 1.0

Для 200 самых дешевых домов:
Среднее количество комнат (округлено до целог

Вывод:
Влияния на изменение стоимости дорогого жилья при увиличении выборки не несет существенного изменения и колеблется в пределах погрешности
Влияние кол-ва комнат в дешевом жилье существенно изменяется, в зависимости от увеличения размерности выборки

# Задание 5: Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [7]:
query = '''
SELECT LSTAT, MEDV, RANK() OVER (ORDER BY LSTAT) AS LSTAT_Rank
FROM boston
WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
ORDER BY LSTAT
'''
# Запускаем запрос и считываем результаты
df = pd.read_sql(query, con)
print(df)

tasks += 1

    LSTAT  MEDV  LSTAT_Rank
0    1.73  50.0           1
1    1.92  50.0           2
2    2.88  50.0           3
3    2.96  50.0           4
4    2.97  50.0           5
5    3.16  50.0           6
6    3.26  50.0           7
7    3.32  50.0           8
8    3.70  50.0           9
9    3.73  50.0          10
10   4.45  50.0          11
11   4.63  50.0          12
12   5.12  50.0          13
13   7.44  50.0          14
14   8.88  50.0          15
15   9.53  50.0          16


Вывод: самые дорогие районы не являются монолитными с точки зрения социально-экономического статуса населения. Данные показывают наличие домов в этих районах, где проживает население с более низким социально-экономическим статусом. Это может свидетельствовать о сложной социальной структуре города и разнообразии жилищных условий, доступных в рамках одной ценовой категории. Такие результаты могут представлять интерес для исследователей городской среды, социологов и разработчиков городской политики, поскольку показывают важность комплексного подхода к анализу жилищных условий и населения городов.

# Задание 6: Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [8]:
query_5 = '''
SELECT 
    CHAS,
    AVG(MEDV) as AVG_MEDV
FROM 
    boston
GROUP BY
    CHAS
'''

# Выполняем запрос и выводим результаты
avg_price_df = pd.read_sql(query_5, con)
avg_price_df['Граничит/Не граничит'] = avg_price_df['CHAS'].apply(lambda x: 'Граничит' if x == 1 else 'Не граничит')
avg_price_df['AVG_MEDV'] = avg_price_df['AVG_MEDV'].apply(lambda x: round(x, 2))
print(avg_price_df[['Граничит/Не граничит', 'AVG_MEDV']])
tasks += 1

  Граничит/Не граничит  AVG_MEDV
0          Не граничит     22.09
1             Граничит     28.44


# Задание 7: Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [9]:
query_6 = '''
SELECT 
    'CRIM' AS column_name, AVG(CRIM) AS avg_value_river_side
FROM boston
WHERE CHAS = 1
UNION ALL
SELECT 
    'ZN', AVG(ZN)
FROM boston
WHERE CHAS = 1
UNION ALL
SELECT 
    'INDUS', AVG(INDUS)
FROM boston
WHERE CHAS = 1
UNION ALL
SELECT 
    'NOX', AVG(NOX)
FROM boston
WHERE CHAS = 1
'''
avg_values_river_side = pd.read_sql(query_6, con)

query_6_no_river = '''
SELECT 
    'CRIM' AS column_name, AVG(CRIM) AS avg_value_no_river_side
FROM boston
WHERE CHAS = 0
UNION ALL
SELECT 
    'ZN', AVG(ZN)
FROM boston
WHERE CHAS = 0
UNION ALL
SELECT 
    'INDUS', AVG(INDUS)
FROM boston
WHERE CHAS = 0
UNION ALL
SELECT 
    'NOX', AVG(NOX)
FROM boston
WHERE CHAS = 0
'''
avg_values_no_river_side = pd.read_sql(query_6_no_river, con)

# Слияние двух DataFrame для сравнения средних значений
comparison_df = avg_values_river_side.merge(avg_values_no_river_side, on='column_name')
comparison_df.columns = ['column_name', 'avg_value_river_side', 'avg_value_no_river_side']

# Выбор тех колонок, где среднее значение больше у домов рядом с рекой
higher_avg_river_side = comparison_df[comparison_df['avg_value_river_side'] > comparison_df['avg_value_no_river_side']]

print('Колонки с более высоким средним значением, когда дом граничит с рекой и их средние значения:')
for index, row in higher_avg_river_side.iterrows():
    print(f"{row['column_name']}: {row['avg_value_river_side']:.2f}")

tasks += 1

Колонки с более высоким средним значением, когда дом граничит с рекой и их средние значения:
INDUS: 12.72
NOX: 0.59


Вывод:
1. Промышленная застройка: районы, граничащие с рекой, имеют более высокую долю промышленной застройки по сравнению с районами, не граничащими с рекой. Это может означать, что речные территории привлекают промышленные предприятия по различным причинам, возможно, из-за легкости доступа к воде для промышленного использования или транспортной доступности.

2. Концентрация оксидов азота: Уровень концентрации оксидов азота (NOX), который является показателем загрязнения воздуха, также выше в районах, граничащих с рекой. Это может указывать на то, что промышленные объекты в этих районах вносят значительный вклад в загрязнение воздуха, что особенно важно для рассмотрения при планировании защиты окружающей среды и здоровья общественности.

3. Влияние на экологию и здоровье: Высокие значения NOX могут иметь негативное воздействие на качество воздуха, что, в свою очередь, может негативно сказаться на здоровье жителей этих регионов. Значения INDUS и NOX указывают на более высокую концентрацию промышленной деятельности и связанного с ней загрязнения в районах вблизи реки. 

4. Задачи для местной политики и планирования: Для снижения потенциального негативного воздействия на окружающую среду и здоровье населения важно учитывать эти данные при разработке градостроительных планов и политики в области экологии. Меры могут включать в себя ужесточение экологических стандартов для промышленных предприятий вблизи водных ресурсов, а также развитие зеленых зон и рекреационных территорий, которые могут служить барьером между промышленными и жилыми зонами.

# Задание 8: Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [11]:
query = "SELECT INDUS, NOX FROM boston"
df = pd.read_sql(query, con)

# Функция для расчёта и вывода перцентилей для заданных столбцов
def print_percentiles(df, columns):
    global tasks
    percentiles = np.arange(0.1, 1.1, 0.1) # от 10% до 100% с шагом 10%
    results = {}
    
    for column in columns:
        results[column] = [np.percentile(df[column], p*100) for p in percentiles]
    
    for p in percentiles:
        print(f"Перцентиль {int(p*100)}:")
        for column in columns:
            print(f"{column}: {results[column][int(p*10)-1]}")
        print() # Для лучшей читаемости
    tasks += 1

# Выводим перцентили для INDUS и NOX
print_percentiles(df, ['INDUS', 'NOX'])

Перцентиль 10:
INDUS: 2.91
NOX: 0.427

Перцентиль 20:
INDUS: 4.39
NOX: 0.442

Перцентиль 30:
INDUS: 5.96
NOX: 0.472

Перцентиль 40:
INDUS: 7.38
NOX: 0.507

Перцентиль 50:
INDUS: 9.69
NOX: 0.538

Перцентиль 60:
INDUS: 12.83
NOX: 0.575

Перцентиль 70:
INDUS: 18.1
NOX: 0.605

Перцентиль 80:
INDUS: 18.1
NOX: 0.668

Перцентиль 90:
INDUS: 19.58
NOX: 0.713

Перцентиль 100:
INDUS: 27.74
NOX: 0.871



Вывод:
Основываясь на этих наблюдениях, можно заключить, что между долей промышленной застройки (INDUS) и концентрацией оксидов азота (NOX) существует взаимосвязь. Регионы с высокой долей промышленной застройки склонны иметь и более высокие уровни концентрации оксидов азота, что свидетельствует о потенциальном воздействии промышленности на качество воздуха.

# Проверка количества выполненных заданий в конце кода

In [13]:
if tasks == 8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены не все задания! 🙀


Не понял, почему выполнены не все задания..=(